In [7]:
# check getloader return type supposed to be lr_image, hr_image
from model import SRResnetConfig, LitSRResnet
from dataset import get_dataloader

train_dataset,test_dataset =get_dataloader(batch_size=1, num_workers=3, input_px=128, output_px=262)


Files already downloaded and verified
Files already downloaded and verified


In [8]:
for x, y in train_dataset:
    print(x.shape, y.shape)
    break

torch.Size([1, 3, 128, 128]) torch.Size([1, 3, 262, 262])


In [10]:
# check model architecuture, whethere there are incopatible channel sizes
import torch
from sr.resnet.model import *
config = SRResnetConfig(
    hidden_channel=3,
    #device='cuda',
    lr=1e-3,
    is_training=True
)
model = LitSRResnet(config)
inp = torch.rand(1, 3, 128, 128)*.8#.to('cuda')
target = torch.rand(1, 3, 262, 262)*.8#.to('cuda')
_y = model.model(inp)
print(y.shape)

_y = model.training_step((inp, target), 1)
_y.shape, _y.device=, inp.device=, target.max()=, inp.max()=

torch.Size([1, 3, 262, 262])


(torch.Size([]),
 device(type='cpu'),
 device(type='cpu'),
 tensor(0.8000),
 tensor(0.8000))

In [11]:
import pytorch_lightning as L
from pytorch_lightning.callbacks import ModelCheckpoint


checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',   # Metric to monitor
    filename='model',  # Filename for the best checkpoint
    save_top_k=3,          # Save only the best model
    mode='min'             # Save the model with minimum validation loss
)

# train using lightning
trainer = L.Trainer(max_epochs=2, callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloaders=train_dataset, val_dataloaders=test_dataset)
print(trainer)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type                             | Params | Mode 
-------------------------------------------------------------------
0 | model | SRResnetModelv1                  | 579    | train
1 | pnsr  | PeakSignalNoiseRatio             | 0      | train
2 | ssim  | StructuralSimilarityIndexMeasure | 0      | train
-------------------------------------------------------------------
579       Trainable params
0         Non-trainable params
579       Total params
0.002     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]